In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import os

dir = '/content/drive/MyDrive/Project1/Data/Training/Tables'
files = os.listdir(dir)

In [17]:
for file in files:
  print(file)

seouldata_healthcare_db_annotation.json
seouldata_culture_db_annotation.json
seouldata_publicadministration_db_annotation.json
seouldata_industrialeconomic_db_annotation.json
seouldata_welfare_db_annotation.json
seouldata_climate_db_annotation.json
seouldata_transportation_db_annotation.json
seouldata_education_db_annotation.json
publicdata_publicadministration_db_annotation.json
publicdata_healthcare_db_annotation.json
publicdata_industrialeconomic_db_annotation.json
publicdata_culture_db_annotation.json
publicdata_climate_db_annotation.json
publicdata_welfare_db_annotation.json
publicdata_transportation_db_annotation.json
seoulpopulation_publicadministration_db_annotation.json
publicdata_education_db_annotation.json


In [18]:
import json
import pandas as pd

In [19]:
df = pd.DataFrame()

for filename in os.listdir(dir):
  if filename.endswith('.json'):
    filepath = os.path.join(dir, filename)
    with open(filepath, 'r') as f:
      data = json.load(f)
      df_temp = pd.DataFrame(data["data"])
      df = pd.concat([df, df_temp])

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5121 entries, 0 to 139
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   source                 5121 non-null   object
 1   db_id                  5121 non-null   object
 2   table_names_original   5121 non-null   object
 3   table_names            5121 non-null   object
 4   column_names_original  5121 non-null   object
 5   column_names           5121 non-null   object
 6   column_types           5121 non-null   object
 7   foreign_keys           5121 non-null   object
 8   primary_keys           5121 non-null   object
dtypes: object(9)
memory usage: 400.1+ KB


In [21]:
def extract_indexes(df):
    """
    DataFrame에서 스키마 정보를 추출합니다.

    Args:
        df: 스키마 정보를 담고 있는 Pandas DataFrame.

    Returns:
        schema_info: 추출된 스키마 정보를 담은 리스트.
    """
    schema_info = []
    db_ids = {}

    for index, row in df.iterrows():
        schema_info.append({
            "db_id": row['db_id'],
            "table_names_original": row['table_names_original'],
            "table_names": row['table_names'],
            "column_names_original": row['column_names_original'],
            "column_names": row['column_names'],
            "column_types": row['column_types'],
        })

    for index, row in df.iterrows():
      db_ids[row['db_id']] = row['table_names_original'][0]

    return schema_info, db_ids

In [22]:
# DataFrame에서 스키마 정보 추출
schema_info, db_ids = extract_indexes(df)

In [23]:
# 추출된 정보 출력
for schema in schema_info:
    print(schema)

Output hidden; open in https://colab.research.google.com to view.

In [24]:
def print_db_ids(db_ids):
  for db_id, table_names in db_ids.items():
    print(f"db_id: {db_id}, table_names_original: {table_names}")

# db_ids 출력
print_db_ids(db_ids)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
db_id: seouldata_healthcare_3923, table_names_original: L_O_C_A_L_D_A_T_A_072202_N_W
db_id: seouldata_healthcare_1862, table_names_original: L_O_C_A_L_D_A_T_A_072404_J_G
db_id: seouldata_healthcare_3986, table_names_original: L_O_C_A_L_D_A_T_A_072101_Y_S
db_id: seouldata_healthcare_3153, table_names_original: L_O_C_A_L_D_A_T_A_072210_Y_S
db_id: seouldata_healthcare_3043, table_names_original: L_O_C_A_L_D_A_T_A_051801_S_C
db_id: seouldata_healthcare_4081, table_names_original: L_O_C_A_L_D_A_T_A_072405_J_G
db_id: seouldata_healthcare_4139, table_names_original: L_O_C_A_L_D_A_T_A_072301_S_C
db_id: seouldata_healthcare_3518, table_names_original: L_O_C_A_L_D_A_T_A_072208_S_P
db_id: seouldata_healthcare_540, table_names_original: L_O_C_A_L_D_A_T_A_072212
db_id: seouldata_healthcare_871, table_names_original: L_O_C_A_L_D_A_T_A_072215
db_id: seouldata_healthcare_1427, table_names_original: L_O_C_A_L_D_A_T_A_020301_S_B
db_id: seouldata_healthcare_403, table_

In [25]:
with open("/content/drive/MyDrive/Project1/Data/Training/Tables2/values.json", "r", encoding="utf-8") as json_file:
    values_dict = json.load(json_file)

In [26]:
for db_id, values in values_dict.items():
    print(f"db_id: {db_id}, values: {values}")

Output hidden; open in https://colab.research.google.com to view.

In [27]:
!pip install sentence-transformers
!pip install faiss-cpu

In [28]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 임베딩 모델 로드
model = SentenceTransformer('nlpai-lab/KoE5')

# 스키마 정보를 텍스트로 변환하여 임베딩 생성
schema_texts = [
    f"table_names: {table['table_names_original']}, "
    f"테이블_이름: {table['table_names']}, "
    f"column_names: {', '.join(str(item) for item in table['column_names_original'])}, "
    f"컬럼_이름: {', '.join(str(item) for item in table['column_names'])}, "
    f"column_types: {', '.join(str(item) for item in table['column_types'])}, "
    f"values: {'-'.join(str(item) for item in values_dict[table['db_id']])}"
    for table in schema_info
]

print(schema_texts[0])

schema_embeddings = model.encode(schema_texts)

# FAISS 인덱스 생성 및 벡터 추가
dimension = schema_embeddings.shape[1]  # 임베딩 차원 수
index = faiss.IndexFlatL2(dimension)  # L2 거리 기반 인덱스 생성
index.add(np.array(schema_embeddings))  # 임베딩 추가

print("Finish")

table_names: ['SEOUL_PUBLIC_HYGIENE_BIZ'], 테이블_이름: ['서울시 기타 위생용품 제조업 현황'], column_names: [-1, '*'], [0, 'CGG_CODE'], [0, 'SNT_COB_CODE'], [0, 'YY'], [0, 'UPSO_SNO'], [0, 'SNT_COB_NM'], [0, 'UPSO_GSL_YMD'], [0, 'UPSO_NM'], [0, 'TRDP_AREA'], [0, 'UPSO_SITE_TELNO'], [0, 'BMAN_STDT'], [0, 'BUP_NM'], [0, 'SITE_STDT'], [0, 'ADMDNG_NM'], [0, 'DCB_YMD'], [0, 'ED_FIN_YMD'], [0, 'GAEKSIL'], [0, 'HANSHIL'], [0, 'YANGSIL'], [0, 'CHAIR_NUM'], [0, 'YOKSIL'], [0, 'BALHANSIL_YN'], [0, 'PERM_NT_NO'], [0, 'SITE_ADDR_RD'], 컬럼_이름: [-1, '*'], [0, '시군구코드'], [0, '업종코드'], [0, '년도'], [0, '업소일련번호'], [0, '업종명'], [0, '신고일자'], [0, '업소명'], [0, '영업장면적 제곱미터'], [0, '소재지전화번호'], [0, '영업자시작일'], [0, '법인명'], [0, '소재지시작일'], [0, '행정동명'], [0, '폐업일자'], [0, '위생교육수료일자'], [0, '객실수'], [0, '한실수'], [0, '양실수'], [0, '의자수'], [0, '욕실수'], [0, '발한실'], [0, '허가 신고 번호'], [0, '소재지도로명'], column_types: text, number, number, time, number, text, time, text, number, text, time, text, time, text, time, time, text, text, text, text, text, text, text

In [29]:
# 벡터 크기 확인 (모델의 벡터 크기: 384차원)
print("벡터 크기:", schema_embeddings.shape)  # 결과: (1, 384)

# 첫 번째 문장의 벡터 일부 확인
print("첫 번째 문장의 벡터 값:", schema_embeddings[0][:10])  # 첫 10개 요소 출력

벡터 크기: (5121, 1024)
첫 번째 문장의 벡터 값: [-0.01839815  0.03099487 -0.05068016  0.02091166  0.0354735  -0.04705312
 -0.02485318  0.03923735  0.00743269  0.05867703]


In [30]:
# 사용자 질문 예시
query = "이름에 교통행정과가 들어가는 기관의 주소를 알려줘"
query_embedding = model.encode([query])

# FAISS 인덱스에서 유사한 스키마 검색
k = 5  # 상위 2개의 관련 결과를 검색
distances, indices = index.search(np.array(query_embedding), k)

print("가장 관련성이 높은 스키마 정보:")
for idx in indices[0]:
    print(schema_texts[idx])

가장 관련성이 높은 스키마 정보:
table_names: ['ICTR_INCHEON_SUBWAY_STATION_ADDRESS_PHONE_NUM'], 테이블_이름: ['인천교통공사 인천지하철 역 주소 및 전화번호'], column_names: [-1, '*'], [0, 'DIVISION'], [0, 'STATION_NAME'], [0, 'PHONE_NUMBER'], [0, 'ADDRESS'], 컬럼_이름: [-1, '*'], [0, '구분'], [0, '역명'], [0, '전화번호'], [0, '주 소'], column_types: text, text, text, text, text, values:  '1호선', '부평역', '032-****-****', '인천광역시 부평구 광장로 지하 15' - '1호선', '송도달빛축제공원역', '032-****-****', '인천광역시 연수구 송도동 340' - '1호선', '인천시청역', '032-****-****', '인천광역시 남동구 예술로 지하 264' - '2호선', '가정중앙시장역', '032-****-****', '인천광역시 서구 가남로 400 (가정동 )'- '2호선', '아시아드경기장역', '032-****-****', '인천광역시 서구 서곶로 382 (연희동 )'- '2호선', '주안역', '032-****-****', '인천광역시 미추홀구 주안로 지하 91-1 (주안동 )'- '7호선', '신중동역', '032-****-****', '경기도 부천시 원미구 길주로 지하 314 (중동 )'- '1호선', '캠퍼스타운역', '032-****-****', '인천광역시 연수구 송도국제대로 지하 66' - '2호선', '완정역', '032-****-****', '인천광역시 서구 원당대로 650 (당하동 )'- '1호선', '동막역', '032-****-****', '인천광역시 연수구 경원대로 지하 80' 
table_names: ['ICTR_INCHEON_SUBWAY_STATION_ADDRESS_PHONE_NUM'

In [31]:
# 사용자 질문 예시
query = "기관명과 주소를 알려줘"
query_embedding = model.encode([query])

# FAISS 인덱스에서 유사한 스키마 검색
k = 5  # 상위 2개의 관련 결과를 검색
distances, indices = index.search(np.array(query_embedding), k)

print("가장 관련성이 높은 스키마 정보:")
for idx in indices[0]:
    print(schema_texts[idx])

가장 관련성이 높은 스키마 정보:
table_names: ['KDCA_HUMAN_DERIVED_GOODS_BANK_STS'], 테이블_이름: ['질병관리청 인체유래물은행현황'], column_names: [-1, '*'], [0, 'ANNUAL'], [0, 'LICENSE_NUMBER'], [0, 'KILN'], [0, 'LOCATION'], 컬럼_이름: [-1, '*'], [0, '연번'], [0, '허가번호'], [0, '기 관 명'], [0, '소 재 지'], column_types: text, number, number, text, text, values:  47, 56, '한국원자력의학원', '서울특별시 노원구 노원로 75' - 11, 15, '재단법인 아산사회복지재단 서울아산병원', '서울특별시 송파구 올림픽로43길 88' - 77, 88, '한림대학교 강남성심병원', '서울특별시 영등포구 신길로1' - 15, 20, '가톨릭대학교 성의교정', '서울특별시 서초구 반포대로 222' - 28, 36, '제주대학교병원', '제주특별자치도 제주시 아란13길 15' - 36, 45, '분당서울대학교병원', '경기도 성남시 분당구 구미로 173번길 82' - 38, 47, '한양대학교병원류마티스병원', '서울특별시 성동구 왕십리로 222' - 19, 27, '전북대학교병원', '전라북도 전주시 덕진구 건지로 20' - 26, 34, '순천향대학교 의과대학', '충청남도 천안시 동남구 순천향6길 31' - 51, 60, '양산부산대학교병원', '경상남도 양산시 물금읍 금오로 20' 
table_names: ['KIAT_RESEARCH_EQ_SU_AGENCY'], 테이블_이름: ['한국산업기술진흥원 연구장비지원기관'], column_names: [-1, '*'], [0, 'NOS'], [0, 'NM_ORGANIZATIONT'], [0, 'CENTER_NMU'], [0, 'CENTER_ADDRESSV'], [0, 'CLASW'], 컬럼_이름: [-1, '*'], 

In [32]:
# 사용자 질문 예시
query = "평생학습포털 평생교육기관 현황에서 강서문화원의 홈페이지 주소를 검색해 줄래"
query_embedding = model.encode([query])

# FAISS 인덱스에서 유사한 스키마 검색
k = 5  # 상위 2개의 관련 결과를 검색
distances, indices = index.search(np.array(query_embedding), k)

print("가장 관련성이 높은 스키마 정보:")
for idx in indices[0]:
    print(schema_texts[idx])

가장 관련성이 높은 스키마 정보:
table_names: ['LIFETIME_EDU_ORGAN'], 테이블_이름: ['서울시 평생학습포털 평생교육기관 현황'], column_names: [-1, '*'], [0, 'DEPT_NM'], [0, 'TEL'], [0, 'ADDR'], [0, 'HOMEPAGE_URL'], [0, 'MAP_LAT'], [0, 'MAP_LNG'], 컬럼_이름: [-1, '*'], [0, '기관명'], [0, '기관연락처'], [0, '기관주소'], [0, '홈페이지'], [0, '위치 위도'], [0, '위치 경도'], column_types: text, text, text, text, text, number, text, values:  '서울영어마을수유캠프평생교육시설', '02-****-****', '서울특별시 강북구 수유동 522', 'http://suyu.sev.go.kr/', 37.64025, '127.00817' - '(사 )청년여성문화원부설평생교육원','02-****-****','서울특별시 용산구 대사관로 31길 24 캠퍼스빌딩 2층 4층','http://www.cyw.or.kr',37.53434,'127.00604'- '화곡청소년수련관', '02-****-****', '강서구 곰달래로 57가길 26', 'http://www.hgyouth.or.kr/', NULL, NULL - '덤인평생교육원', '02-****-****', '서울특별시 서초구 강남대로30길 29  2층 3층 (양재동  대동빌딩 )','http://www.dumin.co.kr',37.4822,'127.0388'- '한국커피교육센터', '02-****-****', '서울특별시 용산구 두텁바위로1길 3 신아빌딩 2층 일부 한국커피교육센터', 'http://www.coffeero.com', 37.54541, '126.97244' - '경향씨앤엘평생교육원', '02-****-****', '서울특별시 강북구 덕릉로 147 선경빌딩 2층', 'http://www.khed

In [33]:
# 사용자 질문 예시
query = "서울시 미래 청년 기획단의 기관 주소를 찾아줘"
query_embedding = model.encode([query])

# FAISS 인덱스에서 유사한 스키마 검색
k = 5  # 상위 2개의 관련 결과를 검색
distances, indices = index.search(np.array(query_embedding), k)

print("가장 관련성이 높은 스키마 정보:")
for idx in indices[0]:
    print(schema_texts[idx])

가장 관련성이 높은 스키마 정보:
table_names: ['LIFETIME_EDU_ORGAN'], 테이블_이름: ['서울시 평생학습포털 평생교육기관 현황'], column_names: [-1, '*'], [0, 'DEPT_NM'], [0, 'TEL'], [0, 'ADDR'], [0, 'HOMEPAGE_URL'], [0, 'MAP_LAT'], [0, 'MAP_LNG'], 컬럼_이름: [-1, '*'], [0, '기관명'], [0, '기관연락처'], [0, '기관주소'], [0, '홈페이지'], [0, '위치 위도'], [0, '위치 경도'], column_types: text, text, text, text, text, number, text, values:  '서울영어마을수유캠프평생교육시설', '02-****-****', '서울특별시 강북구 수유동 522', 'http://suyu.sev.go.kr/', 37.64025, '127.00817' - '(사 )청년여성문화원부설평생교육원','02-****-****','서울특별시 용산구 대사관로 31길 24 캠퍼스빌딩 2층 4층','http://www.cyw.or.kr',37.53434,'127.00604'- '화곡청소년수련관', '02-****-****', '강서구 곰달래로 57가길 26', 'http://www.hgyouth.or.kr/', NULL, NULL - '덤인평생교육원', '02-****-****', '서울특별시 서초구 강남대로30길 29  2층 3층 (양재동  대동빌딩 )','http://www.dumin.co.kr',37.4822,'127.0388'- '한국커피교육센터', '02-****-****', '서울특별시 용산구 두텁바위로1길 3 신아빌딩 2층 일부 한국커피교육센터', 'http://www.coffeero.com', 37.54541, '126.97244' - '경향씨앤엘평생교육원', '02-****-****', '서울특별시 강북구 덕릉로 147 선경빌딩 2층', 'http://www.khed

In [34]:
def merge_json_files_to_dataframe(root_dir):
    """
    Merges all JSON files in subfolders of the root directory into a single DataFrame.

    Args:
        root_dir: The path to the root directory.

    Returns:
        A Pandas DataFrame containing the merged data.
    """
    df = pd.DataFrame()  # Create DataFrame

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.json'):
                filepath = os.path.join(dirpath, filename)
                print(filepath)
                with open(filepath, 'r', encoding='utf-8') as f:
                    try:
                        data = json.load(f)
                        df_temp = pd.DataFrame(data["data"])
                        df = pd.concat([df, df_temp])
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON in file: {filepath}")
                        print(f"Error message: {e}")

    return df

In [35]:
# 구글 드라이브 폴더 경로를 지정합니다.
folder_path = '/content/drive/MyDrive/Project1/Data/Training/SQLs'

# JSON 파일을 읽어와 병합합니다.
df2 = merge_json_files_to_dataframe(folder_path)

# 병합된 DataFrame을 출력합니다.
print(df2.info())

/content/drive/MyDrive/Project1/Data/Training/SQLs/2. 공공데이터포털/01. 보건/TEXT_NL2SQL_label_publicdata_healthcare.json
/content/drive/MyDrive/Project1/Data/Training/SQLs/2. 공공데이터포털/03. 문화／관광/TEXT_NL2SQL_label_publicdata_culture.json
/content/drive/MyDrive/Project1/Data/Training/SQLs/2. 공공데이터포털/08. 교육/TEXT_NL2SQL_label_publicdata_education.json
/content/drive/MyDrive/Project1/Data/Training/SQLs/2. 공공데이터포털/06. 환경/TEXT_NL2SQL_label_publicdata_climate.json
/content/drive/MyDrive/Project1/Data/Training/SQLs/2. 공공데이터포털/02. 일반행정/TEXT_NL2SQL_label_publicdata_publicadministration.json
/content/drive/MyDrive/Project1/Data/Training/SQLs/2. 공공데이터포털/04. 산업／경제/TEXT_NL2SQL_label_publicdata_industrialeconomic.json
/content/drive/MyDrive/Project1/Data/Training/SQLs/2. 공공데이터포털/05. 복지/TEXT_NL2SQL_label_publicdata_welfare.json
/content/drive/MyDrive/Project1/Data/Training/SQLs/2. 공공데이터포털/07. 교통

In [36]:
df2 = df2.dropna(subset=['query'])

df2.info()
df2.head()

<class 'pandas.core.frame.DataFrame'>
Index: 88946 entries, 0 to 310
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   db_id           88946 non-null  object
 1   utterance_id    88946 non-null  object
 2   hardness        88946 non-null  object
 3   utterance_type  88946 non-null  object
 4   query           88946 non-null  object
 5   utterance       88946 non-null  object
 6   values          88946 non-null  object
 7   cols            88946 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


,db_id,utterance_id,hardness,utterance_type,query,utterance,values,cols
0,publicdata_healthcare_1812,Wht_9888,medium,BR04,SELECT DISTINCT CONTACT FROM FOOD_RELATED_INDU...,소재지 도로명에 무내미길이 들어가는 업소의 연락처를 중복 없이 보여줘,[],"[{'token': '연락처', 'start': 24, 'column_index':..."
1,publicdata_healthcare_1812,Whr_12275,hard,BR03,"SELECT LATITUDE, HARDNESS FROM FOOD_RELATED_IN...",소재지 도로명 지평면이 들어가며 업소명에 회사가 포함되는 곳의 위도와 경도를 알려줘,"[{'token': '지평면', 'start': 8, 'column_index': ...","[{'token': '경도', 'start': 39, 'column_index': ..."
2,publicdata_healthcare_1812,Wht_9889,easy,BR04,SELECT INDUSTRY_NAME FROM FOOD_RELATED_INDUSTR...,이름이 알이네장터인 업소의 업종명은 뭐야,"[{'token': '알이네장터', 'start': 4, 'column_index'...","[{'token': '업종명', 'start': 15, 'column_index':..."
3,publicdata_healthcare_1812,Whr_12276,easy,BR03,SELECT DISTINCT MATERIAL_MAP_NAME FROM FOOD_RE...,라라마트의 소재지 도로명을 중복 없이 보여줘,"[{'token': '라라마트', 'start': 0, 'column_index':...","[{'token': '소재지 도로명', 'start': 6, 'column_inde..."
4,publicdata_healthcare_1812,Whr_12277,easy,BR03,SELECT HARDNESS FROM FOOD_RELATED_INDUSTRIES_Y...,연락처가 031-772-0189인 곳의 경도를 찾아줘,"[{'token': '031-772-0189', 'start': 5, 'column...","[{'token': '경도', 'start': 22, 'column_index': 5}]"


In [37]:
def check_db(query, db_id, n):
  query_embedding = model.encode([query])

  # FAISS 인덱스에서 유사한 스키마 검색
  k = n  # 상위 n개의 관련 결과를 검색
  distances, indices = index.search(np.array(query_embedding), k)

  for idx in indices[0]:
      if db_ids[db_id] in schema_texts[idx]:
          return True

  return False

In [38]:
# prompt: df2에서 db_id와 utterance 컬럼의 값들을 리스트 쌍으로 가져오는 코드를 작성해주세요

db_utterance_pairs = list(zip(df2['db_id'].tolist(), df2['utterance'].tolist()))

print(db_utterance_pairs[0])

('publicdata_healthcare_1812', '소재지 도로명에 무내미길이 들어가는 업소의 연락처를 중복 없이 보여줘')


In [39]:
# k = 1

count_all = 0
count_true = 0
count_false = 0

for q in db_utterance_pairs:
  if check_db(q[1], q[0], 1) == True:
    count_true += 1
  else:
    count_false += 1
  count_all += 1

print(count_all)
print(count_true)
print(count_false)

88946
22785
66161


In [40]:
# k = 5

count_all = 0
count_true = 0
count_false = 0

for q in db_utterance_pairs:
  if check_db(q[1], q[0], 5) == True:
    count_true += 1
  else:
    count_false += 1
  count_all += 1

print(count_all)
print(count_true)
print(count_false)

88946
38229
50717
